## **38.9 PROJECT. Исследование поведения пользователей**
<font size="5" color="red"><b>DAPR-1016

 Pidenko Dmitry</b><font>


 **ЛЕГЕНДА**

      Продолжаем работать продуктовым аналитиком в гейм-студии.

Новый день — новые вызовы! На этот раз  будем разбираться с мобильной игрой Quiz Freeze. Это сборник вопросов в различных форматах и темах, на которые игрокам нужно давать ответы. Игра одиночная, то есть каждый игрок отвечает на вопросы самостоятельно.
Некоторое время назад игру основательно переделали, но хороший и качественный анализ изменений, к сожалению, до сих пор не провели. Надо это исправить! Собравшись с командой, мы определили задачи, которые предстоит решить.

 **ЦЕЛЬ**

Исследовать поведение пользователей в обновлённом приложении.

**ЗАДАЧИ**

      - есть ли зависимость между выбранным уровнем сложности и вероятностью оплаты;
      - различается ли временной промежуток между регистрацией и оплатой у групп пользователей с разным уровнем сложности.

## **Данные содержатся в двух таблицах**
### **1. Загружаем таблицы c данными формата "csv"**

In [16]:
import pandas as pd 
# events_df - Хранит данные о событиях, которые совершают пользователи.
events_df = pd.read_csv('data/7_4_Events.csv', sep = ',')
# purchse_df - Хранит данные об оплатах, которые совершают пользователи.
purchase_df = pd.read_csv('data/purchase.csv', sep = ',')

In [17]:
# Копируем данные на тот случай, если что то пойдет не так
events_df = events_df.copy()
purchase_df = purchase_df.copy() 

In [18]:
# Выведем по 10 строк датафрэймов для ознакомления с данными
display(events_df.head(10))
display(purchase_df.head(10))

,id,event_type,selected_level,start_time,tutorial_id,user_id
0,28903,registration,NaN,2016-05-11T23:40:55,NaN,12583
1,28904,registration,NaN,2016-05-11T23:49:58,NaN,12584
2,28905,registration,NaN,2016-05-12T00:53:07,NaN,12585
3,28906,tutorial_start,NaN,2016-05-12T01:32:20,17562.0,12585
4,28907,tutorial_finish,NaN,2016-05-12T01:34:53,17562.0,12585
5,28908,tutorial_start,NaN,2016-05-12T02:11:29,17563.0,12584
6,28909,tutorial_finish,NaN,2016-05-12T02:14:14,17563.0,12584
7,28910,level_choice,hard,2016-05-12T05:30:25,NaN,12584
8,28911,pack_choice,NaN,2016-05-12T05:38:24,NaN,12584
9,28912,tutorial_start,NaN,2016-05-12T07:19:01,17564.0,12583


,id,user_id,event_datetime,amount
0,15674,12584,2016-05-12T10:34:16,100
1,15675,12985,2016-05-13T08:25:56,50
2,15676,12828,2016-05-13T16:33:46,50
3,15677,12598,2016-05-14T01:09:37,150
4,15678,13037,2016-05-14T01:24:46,100
5,15679,12989,2016-05-14T02:19:20,200
6,15680,12684,2016-05-14T03:10:14,100
7,15681,12656,2016-05-14T03:42:48,100
8,15682,12624,2016-05-14T06:00:45,50
9,15683,13136,2016-05-14T07:31:03,50


In [19]:
# Подробная информация о таблицах
events_df.info()
purchase_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252334 entries, 0 to 252333
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              252334 non-null  int64  
 1   event_type      252334 non-null  object 
 2   selected_level  31086 non-null   object 
 3   start_time      252334 non-null  object 
 4   tutorial_id     125103 non-null  float64
 5   user_id         252334 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 11.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5956 entries, 0 to 5955
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              5956 non-null   int64 
 1   user_id         5956 non-null   int64 
 2   event_datetime  5956 non-null   object
 3   amount          5956 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 186.2+ KB


## **Задания будем выполнять с использованием данных пользователей, зарегистрировавшихся в 2018 году (с 1 января по 31 декабря 2018 года включительно).** 
### **2. Скорректируем данные под наше требование**

In [ ]:
# Формируем маски для выборки пользователей, зарегистрированных в 2018 году
mask1 = events_df['start_time'] >= '2018-01-01' 
mask2 = events_df['start_time'] < '2019-01-01'
mask3 = events_df['event_type'] == 'registration'
# Урезаем загруженные датасеты, оставляя только данные за 2018 год
users2018 = events_df[mask1&mask2&mask3]['user_id'].to_list()
events_df = events_df[events_df['user_id'].isin(users2018)]
purchase_df = purchase_df[purchase_df['user_id'].isin(users2018)]

### **Оценим характеристики каждого столбца после корректировки датасетов**

In [25]:
display(events_df.describe(include='all'))
display(purchase_df.describe(include='all'))

,id,event_type,selected_level,start_time,tutorial_id,user_id
count,66959.000000,66959,8342,66959,32954.000000,66959.000000
unique,NaN,5,3,66809,NaN,NaN
top,NaN,registration,medium,2018-03-13T09:37:43,NaN,NaN
freq,NaN,19926,4645,9,NaN,NaN
mean,113787.000045,NaN,NaN,NaN,40532.934393,37781.543362
std,19329.542752,NaN,NaN,NaN,5213.486632,5751.497904
min,80308.000000,NaN,NaN,NaN,31505.000000,27832.000000
25%,97047.500000,NaN,NaN,NaN,36008.250000,32849.000000
50%,113787.000000,NaN,NaN,NaN,40529.500000,37719.000000
75%,130526.500000,NaN,NaN,NaN,45057.750000,42733.000000


,id,user_id,event_datetime,amount
count,1600.000000,1600.000000,1600,1600.000000
unique,NaN,NaN,1600,NaN
top,NaN,NaN,2018-01-03T18:53:43,NaN
freq,NaN,NaN,1,NaN
mean,17645.505625,37752.766250,NaN,110.734375
std,462.038637,5822.621784,NaN,54.696628
min,16845.000000,27845.000000,NaN,25.000000
25%,17245.750000,32815.750000,NaN,50.000000
50%,17645.500000,37633.500000,NaN,100.000000
75%,18045.250000,43023.000000,NaN,150.000000


## **Так как данные содержатся в двух таблицах, целесообразно будет если мы объедим их воедино**

### **3. Объединяем датафреймы и присваиваем  имя <font size="5" color="red"><b>total_events_df</b> </font>**

In [26]:
# Добавим в датафрейм purchase_df столбец event_type, который будет содержать одно значение - purchase.
# Это нужно, для того чтобы в объединённом датафрейме однозначно выделить события оплаты.
purchase_df['event_type'] = 'purchase'

In [27]:
# Переименуем столбцы id, которые несут различный смысл в исходных датасетах.
# Приведем поле даты события к единому наименованию - start_time  
events_df = events_df.rename(columns={'id': 'event_id'})
purchase_df = purchase_df.rename(columns={
        'id': 'purchase_id', 
        'event_datetime': 'start_time'
    })

In [42]:
# Формируем объединенную таблицу событий - total_events_df
total_events_df = pd.concat([events_df,purchase_df],sort=False)
# Приведем поле start_time к типу DateTime:
total_events_df['start_time'] = pd.to_datetime(total_events_df['start_time'])
# Проведем сброс индексов после объединения, отсортируем события по времени
total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')
# Проверим содержимое полученной таблицы
total_events_df.describe(include='all')

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,amount
count,66959.000000,68559,8342,68559,32954.000000,68559.000000,1600.000000,1600.000000
unique,NaN,6,3,NaN,NaN,NaN,NaN,NaN
top,NaN,registration,medium,NaN,NaN,NaN,NaN,NaN
freq,NaN,19926,4645,NaN,NaN,NaN,NaN,NaN
mean,113787.000045,NaN,NaN,2018-07-03 09:04:03.183914496,40532.934393,37780.871775,17645.505625,110.734375
min,80308.000000,NaN,NaN,2018-01-01 03:48:40,31505.000000,27832.000000,16845.000000,25.000000
25%,97047.500000,NaN,NaN,2018-03-24 12:54:39,36008.250000,32848.500000,17245.750000,50.000000
50%,113787.000000,NaN,NaN,2018-06-23 18:58:38,40529.500000,37717.000000,17645.500000,100.000000
75%,130526.500000,NaN,NaN,2018-10-12 13:25:59.500000,45057.750000,42742.500000,18045.250000,150.000000
max,147267.000000,NaN,NaN,2019-01-06 07:14:25,49554.000000,47757.000000,18452.000000,300.000000


### Далее сформируем таблицы, уточняющие принадлежность пользователей:

 <font size="5" color="red"><b>user_diff_lvl</b> </font> - пользователи, производившие выбор уровня сложности 

 <font size="5" color="red"><b>user_add_pay</b> </font> - пользователи, производившие оплату дополнительного контента 
 
 <font size="5" color="red"><b>user_reg</b> </font> - пользователи, прошедшие регистрацию 

In [52]:
user_diff_lvl = total_events_df[total_events_df['event_type'] == 'level_choice']\
    [['user_id','selected_level','start_time']].rename(
        columns={'start_time':'sel_time'}
    )

user_add_pay = total_events_df[total_events_df['event_type'] == 'purchase']\
    [['user_id','start_time']].rename(
        columns={'start_time':'pay_time'}
    )

user_reg = total_events_df[total_events_df['event_type'] == 'registration']\
    [['user_id','start_time']].rename(
        columns={'start_time':'reg_time'}
    )

### **4. Расчет показателей**

In [59]:
# Создадим датафрейм, объединяющий информацию о выбранных уровнях и времени оплаты пользователей. 
time_level_and_pay = user_diff_lvl[['user_id','selected_level']].merge(
    user_add_pay, 
    on='user_id',
    how='left'
    )

# Рассчитаем показатели <<Статистика оплат по выбранным уровням сложности>>
pay_by_diff_lvl = (
    time_level_and_pay.groupby(by='selected_level')['pay_time'].agg(['count'])*100 / 
    user_diff_lvl.groupby(by='selected_level')['user_id'].agg(['count'])
)
print('Процент оплативших доп. контент, в зависимости от выбранного  уровня сложности:')
display(round( pay_by_diff_lvl.sort_values(by='count')))

Процент оплативших доп. контент, в зависимости от выбранного  уровня сложности:


,count
selected_level,
easy,8.0
medium,21.0
hard,35.0


### **Произведем расчет средних значений временного интервала между регистрацией пользователя и проведенной оплатой, по каждому выбранному уровню сложности.**

In [66]:
# Создадим датафрейм, объединяющий информацию о времени регистрации и времени оплаты пользователей. 
regtime_vs_purchase = user_reg.merge(
    time_level_and_pay, 
    on='user_id',
    how='inner'
    )

# В полученном датафрейме создадим поле timedelta, где рассчитан временной интервал "регистрация-покупка"
regtime_vs_purchase['timedelta'] = (
    regtime_vs_purchase['pay_time'] -
    regtime_vs_purchase['reg_time']
)

# Рассчитаем показатели <<Среднее время от регистрации до оплаты, по выбранным уровням сложности>>
paytime_from_reg_by_levl = regtime_vs_purchase.groupby(
    by='selected_level'
    )['timedelta'].mean()

print('Среднее время от регистрации до оплаты, по выбранным уровням сложности:')
display(paytime_from_reg_by_levl.sort_values())

Среднее время от регистрации до оплаты, по выбранным уровням сложности:


selected_level
hard     3 days 14:55:19.257918552
easy     3 days 22:10:23.211640211
medium   4 days 06:12:06.576883384
Name: timedelta, dtype: timedelta64[ns]

### Найдем среднее время от выбора уровня сложности до оплаты.

In [68]:
# Создадим датафрейм, объединяющий информацию о времени выбора уровня и времени оплаты пользователей.
diftime_vs_paytime = user_diff_lvl.merge(
    user_add_pay,
    on='user_id',
    how='inner'
    )

# В полученном датафрейме создадим поле timedelta, где рассчитан временной интервал "выбор уровня - покупка"
diftime_vs_paytime['timedelta'] = (diftime_vs_paytime['pay_time'] - 
    diftime_vs_paytime['sel_time']
)

# Рассчитаем показатели <<Среднее время от выбора уровня до оплаты, по выбранным уровням сложности>>
paytime_from_diff_by_levl = diftime_vs_paytime.groupby(
    by='selected_level'
    )['timedelta'].mean()

print('Среднее время от выбора уровня сложности до оплаты, по уровням сложности:')
display(paytime_from_diff_by_levl.sort_values())

Среднее время от выбора уровня сложности до оплаты, по уровням сложности:


selected_level
hard     3 days 07:20:41.420814479
easy     3 days 14:58:52.941798941
medium   3 days 23:14:13.165118679
Name: timedelta, dtype: timedelta64[ns]

### **5. Вывод**

<font size="5" color="red">Есть ли зависимость между выбранным уровнем сложности и вероятностью оплаты?</font>
 
 Зависимость между выбранным уровнем сложности и вероятностью оплаты, есть, при чем большая. Чем выше уровень сложности, тем больше вероятность, что пользователь приобретет дополнительный пакет вопросов:<br>
<pre>easy     8%
medium   21%
hard     35%</pre>




<font size="5" color="red">Различается ли временной промежуток между регистрацией и оплатой у групп пользователей с разным уровнем сложности?</font>

На основании полученных средних интервалов времени от регистрации пользователей до оплаты, не удалось найти значительных различий. Полученные результаты отличаются не значительно:
<pre>hard     3 дня 14 часов 55 минут
easy     3 дня 22 часа 10 минут 
medium   4 дня 06 часов 12 минут </pre>